In [1]:
import ee
import geemap
import datetime

# Inicjalizacja GEE
ee.Initialize(project='ee-jsumara')

In [2]:
# Parametry
start_date = '2023-04-01'
end_date = '2023-07-31'

# Obszar zainteresowania – Lublin (przybliżony bbox)
lublin_aoi = ee.Geometry.Rectangle([22.45, 51.18, 22.65, 51.30])


In [6]:
# Sentinel-2 (maskowanie chmur + NDVI)
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000)

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterDate(start_date, end_date)
      .filterBounds(lublin_aoi)
      .map(mask_clouds)
      .select(['B4', 'B8']))  # Czerwony i NIR do NDVI

def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

s2 = s2.map(add_ndvi)

In [7]:
# Sentinel-1 (VV i VH)
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
      .filterDate(start_date, end_date)
      .filterBounds(lublin_aoi)
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .select(['VV', 'VH']))


In [8]:
# Fuzja danych (średnia kompozycja)
s2_median = s2.median()
s1_median = s1.median()
fused = s2_median.addBands(s1_median)

In [9]:
# Podgląd
Map = geemap.Map(center=[51.24, 22.55], zoom=12)
Map.addLayer(fused, {
    'bands': ['NDVI', 'VV', 'VH'],
    'min': [-0.2, -20, -25],
    'max': [0.9, 0, 0],
}, 'Fused Image (NDVI + S1 VV/VH)')
Map

Map(center=[51.24, 22.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…